In [155]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from random import randint
from numpy import array, argmax, asarray, zeros
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
import spacy
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
nlp=spacy.load("en")
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import LSTM, Embedding,GRU
from keras.models import Sequential
from keras.layers import Flatten, Masking
from keras.layers.recurrent import SimpleRNN, LSTM

## define function

In [315]:
lem = WordNetLemmatizer()
def lem_sentences(sentence):
    tokens = nltk.word_tokenize(sentence)
    lemmed_tokens = [lem.lemmatize(token) for token in tokens]
    return ' '.join(lemmed_tokens)

def integer_encode_documents(docs,tokenizer):
    return tokenizer.texts_to_sequences(docs)

## data preparation

In [317]:
tag_data = pd.read_csv('tagged_product_attributes.csv')
full_data = pd.read_csv('full_data.csv')


In [318]:
## pick all of tagged 'embellishment' data 
tag_data['attribute_value']=tag_data['attribute_value'].str.lower()
embellishment=tag_data[tag_data['attribute_name'].str.lower()=='embellishment']

In [319]:
## drop duplicate
embellishment_1=embellishment.drop_duplicates(subset=['product_id','attribute_value'])
embellishment_2=pd.concat([embellishment_1,pd.get_dummies(embellishment_1['attribute_value'])],axis=1)
#embellishment_2['attribute_value'].value_counts()
embellishments=list(embellishment_2['attribute_value'].value_counts().index)


In [320]:
#prepare trained data
full=full_data[['product_id','brand','brand_category','product_full_name','description','details']]\
.drop_duplicates(subset=['product_id'])
full=full.fillna('')
full['input']=full[['brand','brand_category','product_full_name','description','details']].agg(' '.join, axis=1).str.lower()
full['input'] = full['input'].replace(r"[^a-z0-9\s]+", "")
embellishment_tag=embellishment_2.merge(full,how='left',on='product_id')
embellishment_tag=embellishment_tag.groupby(['product_id','input'])[embellishments].sum().reset_index()
embellishment_tag['input']=embellishment_tag['input'].apply(lambda x: lem_sentences(x))
embellishment_tag.head()


,product_id,input,ruffles,studs,embroidery,trim,buckles,beaded,lace,mesh,fringe,sequins,rhinestone,crystals,patches,epaulets
0,01DPCG1C1P0MQAV9NMS3N1TDAA,j.crew unknown collection fluted sheath dress ...,0,0,1,0,0,0,0,1,0,0,0,0,0,0
1,01DPD1R0F6Z4EJCPNDKPAG5DPF,veronica beard unknown lebone pant our lebone ...,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,01DPD2FGA0FWFXJCKMT5QBJYHK,veronica beard unknown ravello skirt in our pl...,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,01DPD3AGTNRNAKNP8TH9WYG35X,veronica beard jacket theron jacket our theron...,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,01DPD4R5X5TQCWTVTC2AEAFC10,veronica beard dress ida dress our ida floral ...,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Load in GloVe Vectors

In [321]:
def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


##  Define and Fit Model

In [349]:
#try ruffles

inputdata=embellishment_tag['input'].tolist()
label=np.array(embellishment_tag['ruffles'].tolist())
stopwords_removed_docs = list(
    map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
tokenizer.fit_on_texts(stopwords_removed_docs)
encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
padded_docs = pad_sequences(encoded_docs,padding='post')
labels = to_categorical(encoder.fit_transform(label))
#padded_docs


In [350]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2,random_state=0)

In [351]:
VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
VOCAB_SIZE 

32412

In [352]:
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [353]:
def make_binary_classification_rnn_model():
    model = Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix],trainable=False))
    model.add(Masking(mask_value=0.0)) 
    model.add(SimpleRNN(units=64))
    model.add(Dense(16))
    model.add(Dense(2, activation='softmax'))
    model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

In [354]:
model = make_binary_classification_rnn_model()


In [355]:
history = model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 5ms/step - loss: 0.6117 - accuracy: 0.7161 - val_loss: 0.5357 - val_accuracy: 0.7778
Epoch 2/5
236/236 [==============================] - 1s 2ms/step - loss: 0.4769 - accuracy: 0.7712 - val_loss: 0.4648 - val_accuracy: 0.8148
Epoch 3/5
236/236 [==============================] - 1s 3ms/step - loss: 0.3861 - accuracy: 0.8305 - val_loss: 0.4558 - val_accuracy: 0.8889
Epoch 4/5
236/236 [==============================] - 1s 2ms/step - loss: 0.3434 - accuracy: 0.8517 - val_loss: 0.4166 - val_accuracy: 0.8889
Epoch 5/5
236/236 [==============================] - 1s 2ms/step - loss: 0.2965 - accuracy: 0.8814 - val_loss: 0.3998 - val_accuracy: 0.9259


## evaluate every tag's RNN performance

In [358]:
import random
random.seed(0)
def value_model(i):
    inputdata=embellishment_tag['input'].tolist()
    label=np.array(embellishment_tag["{}".format(i)].tolist())
    stopwords_removed_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
    tokenizer.fit_on_texts(stopwords_removed_docs)
    encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    labels = to_categorical(encoder.fit_transform(label))
    X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2,random_state=0)
    VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
    embedding_matrix = zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    model = make_binary_classification_rnn_model()
    return model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)

In [359]:
for i in embellishments:
    print(f'{i} prediction performance:')
    value_model(i)

ruffles prediction performance:
Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 5ms/step - loss: 0.6509 - accuracy: 0.6398 - val_loss: 0.5575 - val_accuracy: 0.7778
Epoch 2/5
236/236 [==============================] - 1s 2ms/step - loss: 0.4955 - accuracy: 0.7839 - val_loss: 0.5290 - val_accuracy: 0.8148
Epoch 3/5
236/236 [==============================] - 1s 2ms/step - loss: 0.4289 - accuracy: 0.8178 - val_loss: 0.5529 - val_accuracy: 0.8148
Epoch 4/5
236/236 [==============================] - 1s 2ms/step - loss: 0.3617 - accuracy: 0.8517 - val_loss: 0.4994 - val_accuracy: 0.8148
Epoch 5/5
236/236 [==============================] - 1s 2ms/step - loss: 0.3364 - accuracy: 0.8686 - val_loss: 0.5002 - val_accuracy: 0.8148
studs prediction performance:
Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 6ms/step - loss: 0.5965 - accuracy: 0.6949 - val_loss: 0.4811 - val_accuracy: 0.8148
Ep

## Set model for every tag

In [360]:
# In the following section, our team fit a model for each tag to make a binary prediction whether the record have that type of embellishment
def tag_model(i):
    inputdata=embellishment_tag['input'].tolist()
    label=np.array(embellishment_tag["{}".format(i)].tolist())
    stopwords_removed_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
    tokenizer.fit_on_texts(stopwords_removed_docs)
    encoded_docs = integer_encode_documents(stopwords_removed_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    labels = to_categorical(encoder.fit_transform(label))
    X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2,random_state=0)
    VOCAB_SIZE = int(len(tokenizer.word_index) * 1.1)
    embedding_matrix = zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    model = make_binary_classification_rnn_model()
    model.fit(X_train, y_train,validation_split = 0.1, epochs=5, verbose=1)
    return model


In [361]:
#store all of the trained models in a series for future use
tag_models=pd.Series([tag_model('{}'.format(i)) for i in embellishments],index=embellishments)


Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 6ms/step - loss: 0.7086 - accuracy: 0.6059 - val_loss: 0.6363 - val_accuracy: 0.7407
Epoch 2/5
236/236 [==============================] - 1s 2ms/step - loss: 0.4923 - accuracy: 0.7966 - val_loss: 0.6368 - val_accuracy: 0.7778
Epoch 3/5
236/236 [==============================] - 1s 2ms/step - loss: 0.3989 - accuracy: 0.8263 - val_loss: 0.5979 - val_accuracy: 0.7778
Epoch 4/5
236/236 [==============================] - 1s 3ms/step - loss: 0.3377 - accuracy: 0.8602 - val_loss: 0.5903 - val_accuracy: 0.7778
Epoch 5/5
236/236 [==============================] - 1s 3ms/step - loss: 0.2965 - accuracy: 0.8686 - val_loss: 0.6152 - val_accuracy: 0.8148
Train on 236 samples, validate on 27 samples
Epoch 1/5
236/236 [==============================] - 1s 6ms/step - loss: 0.7545 - accuracy: 0.6271 - val_loss: 0.5226 - val_accuracy: 0.7778
Epoch 2/5
236/236 [==============================] - 1s 2ms/step

## Make Prediction

In [377]:
#define tokenizer
inputdata=embellishment_tag['input'].tolist()
stopwords_removed_docs = list(
map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(stopwords_removed_docs)

In [378]:
description=""##input 
brand=""
brand_category=""
product_full_name=""
details=""
def Identify_embellishment_tag(description, brand, brand_category, product_full_name,details):
    inputdata=[brand,brand_category,product_full_name,description,details]
    inputdata=''.join(inputdata).lower()
    inputdata=inputdata.replace(r"[^a-z0-9\s]+", "")
    inputdata=lem_sentences(inputdata)
    inputdata=[inputdata]
    test_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    encoded_docs = integer_encode_documents(test_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    tags=[]
    for i in embellishments:
        if tag_models[i].predict_classes(padded_docs)==1:
            tags.append(i)
    tags=" ".join(tags)
    print(f'predicted embellishment: {tags}')
    
    
    

In [379]:
#try to test using the row for what we have already known the data
testrow=full.loc[full['product_id']=='01DPCG1C1P0MQAV9NMS3N1TDAA',:]
brand="J.Crew"
brand_category="Unknown"
product_full_name="Collection fluted sheath dress in Ratti® climb..."
details=""
Identify_embellishment_tag(description, brand, brand_category, product_full_name,details)## test is good

predicted embellishment: trim


In [380]:
# make prediction on the full data set
Full_data = pd.read_csv('full_data.csv')
Full_data.head()

,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id
0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,514683,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"A modern pump, in a rounded silhouette with an...","{""Needs Review""}",NaN
1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,526676,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Dress it down with jeans and sneakers or dress...,"{""Needs Review""}",NaN
2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,4.001E+11,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,100% UV protection\nCase and cleaning cloth in...,"{""Needs Review""}",NaN
3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,4.00012E+11,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,Canvas upper\nRound toe\nLace-up vamp\nSmartFO...,"{""Needs Review""}",NaN
4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,4.00011E+11,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,100% UV protection\nGradient lenses\nAdjustabl...,"{""Needs Review""}",NaN


In [381]:
# In the following section, we designed a function that will take input for an item, implement data preprocessing process,
# and record the prediction result generated by models of different tags
def full_embellishment_tag(description, brand, brand_category, product_full_name,details):
    inputdata=[brand,brand_category,product_full_name,description,details]
    inputdata=''.join(inputdata).lower()
    inputdata=inputdata.replace(r"[^a-z0-9\s]+", "")
    inputdata=lem_sentences(inputdata)
    inputdata=[inputdata]
    test_docs = list(
        map(lambda doc: " ".join([token.text for token in nlp(doc) if not token.is_stop]), inputdata))
    encoded_docs = integer_encode_documents(test_docs, tokenizer)
    padded_docs = pad_sequences(encoded_docs, padding='post')
    tags=[]
    for i in embellishments:
        if tag_models[i].predict_classes(padded_docs)==1:
            tags.append(i)
    tags=" ".join(tags)
    return tags

In [384]:
embellishment_output = []
Full_data=Full_data.fillna('')
inputdata = Full_data

for i in range(len(inputdata)):
    description = inputdata.loc[i,"description"]
    brand = inputdata.loc[i,"brand"] 
    brand_category = inputdata.loc[i,"brand_category"] 
    product_full_name = inputdata.loc[i,"product_full_name"] 
    details = inputdata.loc[i,"details"] 
    tags=full_embellishment_tag(description, brand, brand_category, product_full_name,details)
    print(f'{i}:{tags}')
    embellishment_output.append(tags)

0:
1:
2:
3:
4:
5:
6:
7:
8:
9:
10:
11:
12:
13:ruffles
14:
15:ruffles
16:
17:
18:
19:
20:
21:
22:ruffles
23:
24:ruffles
25:
26:
27:
28:
29:
30:
31:
32:
33:
34:
35:ruffles
36:
37:
38:
39:ruffles
40:
41:
42:
43:studs
44:ruffles
45:ruffles
46:
47:
48:fringe
49:
50:
51:
52:
53:
54:
55:
56:
57:
58:ruffles
59:
60:ruffles
61:
62:
63:
64:
65:beaded
66:
67:ruffles
68:ruffles
69:
70:studs
71:
72:
73:
74:
75:
76:
77:
78:
79:
80:
81:
82:
83:
84:trim
85:
86:
87:ruffles
88:
89:crystals
90:
91:
92:
93:
94:
95:
96:ruffles beaded
97:
98:
99:ruffles
100:
101:
102:ruffles
103:
104:
105:
106:
107:ruffles
108:
109:
110:
111:
112:embroidery
113:
114:
115:
116:
117:
118:
119:
120:ruffles
121:
122:
123:ruffles
124:
125:
126:
127:
128:
129:
130:
131:
132:embroidery
133:
134:
135:
136:ruffles
137:
138:
139:ruffles
140:
141:
142:
143:
144:
145:
146:
147:
148:
149:
150:
151:
152:ruffles
153:
154:trim
155:
156:
157:
158:
159:
160:ruffles
161:
162:
163:
164:
165:
166:
167:ruffles
168:ruffles
169:
170:ruffles
171:
172

1229:ruffles
1230:
1231:
1232:ruffles
1233:
1234:
1235:embroidery
1236:
1237:
1238:
1239:
1240:
1241:
1242:
1243:ruffles
1244:
1245:buckles
1246:ruffles
1247:
1248:buckles
1249:
1250:
1251:
1252:
1253:ruffles
1254:
1255:
1256:ruffles
1257:
1258:ruffles
1259:
1260:
1261:
1262:
1263:
1264:ruffles
1265:
1266:
1267:
1268:trim
1269:
1270:
1271:
1272:studs
1273:
1274:ruffles
1275:
1276:ruffles
1277:
1278:ruffles studs embroidery beaded sequins
1279:ruffles
1280:
1281:
1282:
1283:
1284:ruffles
1285:
1286:
1287:
1288:ruffles
1289:
1290:
1291:
1292:ruffles beaded
1293:
1294:
1295:ruffles
1296:
1297:
1298:
1299:
1300:
1301:
1302:buckles
1303:
1304:ruffles
1305:
1306:ruffles
1307:
1308:
1309:
1310:
1311:
1312:
1313:
1314:
1315:ruffles
1316:ruffles
1317:ruffles
1318:
1319:ruffles
1320:
1321:
1322:
1323:studs
1324:
1325:
1326:
1327:
1328:
1329:
1330:
1331:
1332:
1333:studs
1334:ruffles
1335:
1336:
1337:ruffles
1338:
1339:
1340:
1341:studs
1342:
1343:
1344:embroidery
1345:
1346:
1347:
1348:ruffles
1

2295:ruffles
2296:
2297:
2298:
2299:
2300:
2301:
2302:
2303:ruffles
2304:
2305:
2306:
2307:
2308:
2309:
2310:
2311:
2312:
2313:
2314:
2315:
2316:
2317:
2318:
2319:
2320:
2321:
2322:
2323:
2324:
2325:
2326:ruffles
2327:ruffles
2328:
2329:
2330:
2331:
2332:ruffles
2333:
2334:
2335:
2336:ruffles
2337:
2338:studs
2339:studs
2340:
2341:studs
2342:
2343:
2344:
2345:
2346:
2347:
2348:
2349:ruffles
2350:embroidery
2351:
2352:
2353:
2354:
2355:
2356:
2357:ruffles
2358:ruffles
2359:
2360:
2361:
2362:trim
2363:ruffles buckles lace sequins
2364:
2365:embroidery
2366:
2367:studs
2368:
2369:
2370:
2371:ruffles
2372:
2373:
2374:
2375:
2376:
2377:
2378:
2379:
2380:
2381:
2382:
2383:
2384:
2385:
2386:trim
2387:ruffles embroidery beaded
2388:
2389:
2390:
2391:ruffles
2392:
2393:
2394:
2395:
2396:
2397:
2398:
2399:buckles
2400:
2401:embroidery
2402:
2403:
2404:
2405:
2406:
2407:
2408:trim
2409:
2410:
2411:
2412:
2413:embroidery
2414:ruffles
2415:
2416:
2417:
2418:
2419:
2420:
2421:trim lace
2422:
2423:
2

3360:
3361:
3362:
3363:
3364:
3365:
3366:
3367:
3368:
3369:
3370:
3371:
3372:
3373:
3374:
3375:
3376:
3377:
3378:ruffles
3379:
3380:
3381:
3382:buckles
3383:
3384:embroidery
3385:ruffles
3386:ruffles studs embroidery trim fringe
3387:
3388:
3389:
3390:
3391:
3392:sequins
3393:
3394:buckles
3395:
3396:
3397:
3398:
3399:studs
3400:ruffles
3401:
3402:
3403:studs
3404:
3405:
3406:
3407:
3408:
3409:
3410:
3411:
3412:
3413:
3414:
3415:embroidery
3416:
3417:
3418:
3419:
3420:
3421:
3422:
3423:
3424:
3425:
3426:
3427:studs
3428:beaded
3429:
3430:
3431:
3432:
3433:
3434:studs
3435:ruffles
3436:
3437:
3438:
3439:
3440:ruffles sequins
3441:
3442:
3443:
3444:
3445:
3446:
3447:fringe
3448:
3449:
3450:
3451:
3452:
3453:studs
3454:ruffles
3455:trim
3456:
3457:
3458:
3459:
3460:ruffles
3461:
3462:
3463:
3464:
3465:
3466:trim
3467:
3468:
3469:
3470:
3471:
3472:
3473:
3474:
3475:
3476:
3477:
3478:
3479:ruffles
3480:ruffles
3481:ruffles
3482:
3483:
3484:
3485:ruffles
3486:
3487:
3488:studs
3489:
3490:
34

4429:ruffles
4430:
4431:
4432:
4433:
4434:
4435:
4436:embroidery
4437:
4438:
4439:
4440:
4441:
4442:
4443:
4444:
4445:
4446:
4447:
4448:embroidery
4449:
4450:
4451:studs
4452:embroidery
4453:embroidery
4454:
4455:
4456:
4457:trim
4458:
4459:
4460:ruffles studs
4461:
4462:
4463:
4464:ruffles embroidery
4465:
4466:
4467:
4468:
4469:studs sequins
4470:
4471:
4472:
4473:ruffles
4474:buckles
4475:studs beaded
4476:
4477:
4478:
4479:
4480:
4481:
4482:
4483:
4484:
4485:
4486:
4487:
4488:ruffles studs fringe
4489:
4490:
4491:
4492:buckles
4493:
4494:
4495:ruffles
4496:studs
4497:ruffles
4498:ruffles
4499:
4500:
4501:
4502:ruffles
4503:
4504:
4505:
4506:ruffles
4507:
4508:
4509:embroidery trim beaded
4510:
4511:
4512:ruffles studs embroidery trim
4513:
4514:
4515:
4516:
4517:
4518:
4519:
4520:
4521:
4522:
4523:
4524:
4525:
4526:
4527:
4528:
4529:trim
4530:trim
4531:
4532:
4533:
4534:
4535:
4536:
4537:
4538:
4539:
4540:
4541:
4542:
4543:
4544:
4545:
4546:
4547:
4548:
4549:
4550:studs
4551:
4552:

...

45444:
45445:
45446:trim beaded
45447:
45448:
45449:
45450:
45451:
45452:
45453:
45454:
45455:studs
45456:
45457:trim
45458:studs embroidery
45459:ruffles
45460:
45461:
45462:ruffles buckles
45463:
45464:
45465:studs trim
45466:ruffles
45467:
45468:
45469:
45470:
45471:
45472:
45473:
45474:studs
45475:
45476:embroidery
45477:ruffles
45478:
45479:ruffles studs buckles
45480:
45481:ruffles
45482:
45483:
45484:
45485:ruffles
45486:
45487:
45488:buckles beaded
45489:trim lace
45490:
45491:beaded
45492:ruffles
45493:
45494:
45495:
45496:
45497:
45498:ruffles embroidery beaded
45499:ruffles
45500:
45501:ruffles
45502:
45503:
45504:
45505:trim
45506:ruffles studs
45507:trim buckles
45508:
45509:
45510:
45511:trim epaulets
45512:ruffles
45513:
45514:
45515:trim
45516:embroidery beaded
45517:
45518:
45519:ruffles
45520:
45521:
45522:embroidery beaded
45523:
45524:embroidery
45525:
45526:
45527:
45528:ruffles
45529:
45530:
45531:
45532:
45533:studs
45534:
45535:trim
45536:studs
45537:
45538:
455

46299:embroidery
46300:
46301:
46302:ruffles embroidery trim
46303:
46304:
46305:
46306:ruffles
46307:
46308:
46309:
46310:studs
46311:
46312:
46313:ruffles
46314:embroidery trim beaded
46315:
46316:
46317:
46318:
46319:
46320:
46321:
46322:
46323:
46324:
46325:embroidery
46326:
46327:
46328:embroidery
46329:
46330:ruffles
46331:
46332:
46333:
46334:
46335:ruffles beaded
46336:
46337:
46338:studs trim
46339:
46340:
46341:
46342:
46343:
46344:
46345:
46346:studs embroidery
46347:
46348:
46349:sequins
46350:ruffles embroidery trim
46351:
46352:
46353:
46354:
46355:trim
46356:studs
46357:studs
46358:embroidery buckles
46359:
46360:
46361:studs lace
46362:
46363:embroidery
46364:
46365:
46366:sequins
46367:
46368:
46369:
46370:trim
46371:
46372:
46373:
46374:buckles beaded
46375:
46376:
46377:ruffles
46378:
46379:
46380:
46381:
46382:
46383:
46384:ruffles
46385:embroidery
46386:beaded
46387:
46388:ruffles
46389:ruffles trim
46390:
46391:
46392:studs beaded
46393:
46394:
46395:
46396:ruffle

47124:trim
47125:
47126:ruffles
47127:
47128:ruffles
47129:
47130:ruffles
47131:ruffles
47132:
47133:ruffles trim
47134:embroidery trim sequins
47135:
47136:trim
47137:trim
47138:
47139:ruffles
47140:
47141:studs
47142:ruffles
47143:studs
47144:
47145:
47146:
47147:
47148:
47149:
47150:
47151:
47152:
47153:ruffles
47154:
47155:ruffles
47156:
47157:
47158:
47159:
47160:
47161:ruffles embroidery beaded
47162:ruffles
47163:
47164:beaded
47165:
47166:embroidery
47167:lace
47168:ruffles
47169:beaded
47170:
47171:
47172:ruffles
47173:
47174:
47175:
47176:studs
47177:beaded
47178:
47179:studs beaded
47180:
47181:
47182:
47183:ruffles
47184:ruffles
47185:
47186:
47187:
47188:
47189:
47190:
47191:
47192:ruffles
47193:trim
47194:
47195:
47196:
47197:studs trim
47198:
47199:
47200:
47201:ruffles embroidery sequins
47202:ruffles
47203:
47204:
47205:
47206:
47207:
47208:
47209:ruffles
47210:
47211:trim
47212:
47213:
47214:
47215:
47216:
47217:beaded
47218:studs trim
47219:
47220:
47221:
47222:
4722

47980:
47981:
47982:
47983:studs
47984:
47985:
47986:
47987:ruffles
47988:
47989:
47990:ruffles embroidery trim
47991:
47992:ruffles
47993:
47994:
47995:embroidery
47996:ruffles
47997:ruffles
47998:
47999:studs
48000:ruffles
48001:ruffles
48002:ruffles
48003:
48004:trim
48005:ruffles
48006:
48007:
48008:ruffles
48009:
48010:
48011:ruffles
48012:
48013:
48014:
48015:ruffles
48016:
48017:ruffles
48018:
48019:
48020:studs
48021:ruffles
48022:
48023:ruffles
48024:beaded
48025:
48026:
48027:ruffles
48028:ruffles
48029:
48030:
48031:
48032:
48033:
48034:
48035:
48036:
48037:ruffles
48038:
48039:
48040:
48041:ruffles
48042:
48043:studs
48044:ruffles
48045:ruffles
48046:trim
48047:
48048:beaded
48049:
48050:embroidery
48051:
48052:
48053:
48054:
48055:trim
48056:ruffles
48057:
48058:
48059:
48060:
48061:ruffles
48062:ruffles
48063:ruffles
48064:
48065:ruffles
48066:studs
48067:studs
48068:ruffles
48069:studs
48070:ruffles
48071:
48072:
48073:
48074:
48075:studs
48076:studs
48077:
48078:embroid

48886:ruffles
48887:
48888:
48889:ruffles
48890:
48891:
48892:
48893:
48894:ruffles
48895:
48896:
48897:
48898:
48899:
48900:
48901:
48902:
48903:trim
48904:
48905:ruffles
48906:
48907:
48908:
48909:
48910:
48911:beaded
48912:
48913:
48914:studs
48915:
48916:
48917:
48918:
48919:embroidery
48920:
48921:
48922:
48923:
48924:
48925:studs
48926:
48927:
48928:ruffles
48929:
48930:
48931:
48932:
48933:
48934:
48935:
48936:
48937:ruffles
48938:
48939:ruffles
48940:
48941:
48942:ruffles
48943:ruffles
48944:
48945:
48946:ruffles
48947:
48948:
48949:
48950:
48951:
48952:buckles
48953:
48954:embroidery
48955:ruffles
48956:
48957:
48958:
48959:
48960:
48961:
48962:
48963:
48964:
48965:
48966:
48967:
48968:
48969:
48970:
48971:
48972:
48973:
48974:
48975:
48976:
48977:
48978:


In [385]:
embellishment_output

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'ruffles',
 '',
 'ruffles',
 '',
 '',
 '',
 '',
 '',
 '',
 'ruffles',
 '',
 'ruffles',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'ruffles',
 '',
 '',
 '',
 'ruffles',
 '',
 '',
 '',
 'studs',
 'ruffles',
 'ruffles',
 '',
 '',
 'fringe',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'ruffles',
 '',
 'ruffles',
 '',
 '',
 '',
 '',
 'beaded',
 '',
 'ruffles',
 'ruffles',
 '',
 'studs',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'trim',
 '',
 '',
 'ruffles',
 '',
 'crystals',
 '',
 '',
 '',
 '',
 '',
 '',
 'ruffles beaded',
 '',
 '',
 'ruffles',
 '',
 '',
 'ruffles',
 '',
 '',
 '',
 '',
 'ruffles',
 '',
 '',
 '',
 '',
 'embroidery',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'ruffles',
 '',
 '',
 'ruffles',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'embroidery',
 '',
 '',
 '',
 'ruffles',
 '',
 '',
 'ruffles',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'ruffles',
 '',
 'trim',
 '',
 

In [386]:
Full_data['embellishment']=pd.Series(embellishment_output)

In [388]:
Full_data['embellishment'].value_counts()

                                                  37743
ruffles                                            4448
studs                                              1578
trim                                               1058
embroidery                                          788
buckles                                             591
beaded                                              573
fringe                                              188
mesh                                                150
sequins                                             116
lace                                                113
ruffles studs                                       110
ruffles embroidery                                   98
ruffles beaded                                       90
epaulets                                             89
ruffles buckles                                      79
ruffles trim                                         74
studs trim                                      

In [389]:
Full_data.to_csv('full data with embellishment.csv')

In [391]:
pd.read_csv('full data with embellishment.csv')

,Unnamed: 0,product_id,brand,mpn,product_full_name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,bc_product_id,embellishment
0,0,01DSE9TC2DQXDG6GWKW9NMJ416,Banana Republic,514683,Ankle-Strap Pump,"A modern pump, in a rounded silhouette with an...",Unknown,2019-11-11 22:37:15.719107+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"A modern pump, in a rounded silhouette with an...","{""Needs Review""}",NaN,NaN
1,1,01DSE9SKM19XNA6SJP36JZC065,Banana Republic,526676,Petite Tie-Neck Top,Dress it down with jeans and sneakers or dress...,Unknown,2019-11-11 22:36:50.682513+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Dress it down with jeans and sneakers or dress...,"{""Needs Review""}",NaN,NaN
2,2,01DSJX8GD4DSAP76SPR85HRCMN,Loewe,4.001E+11,52MM Padded Leather Round Sunglasses,Padded leather covers classic round sunglasses.,JewelryAccessories/SunglassesReaders/RoundOval...,2019-11-13 17:33:59.581661+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/loewe-52mm-pad...,100% UV protection\nCase and cleaning cloth in...,"{""Needs Review""}",NaN,NaN
3,3,01DSJVKJNS6F4KQ1QM6YYK9AW2,Converse,4.00012E+11,Baby's & Little Kid's All-Star Two-Tone Mid-To...,The iconic mid-top design gets an added dose o...,"JustKids/Shoes/Baby024Months/BabyGirl,JustKids...",2019-11-13 17:05:05.203733+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/converse-babys...,Canvas upper\nRound toe\nLace-up vamp\nSmartFO...,"{""Needs Review""}",NaN,NaN
4,4,01DSK15ZD4D5A0QXA8NSD25YXE,Alexander McQueen,4.00011E+11,64MM Rimless Sunglasses,Hexagonal shades offer a rimless view with int...,JewelryAccessories/SunglassesReaders/RoundOval,2019-11-13 18:42:30.941321+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/alexander-mcqu...,100% UV protection\nGradient lenses\nAdjustabl...,"{""Needs Review""}",NaN,NaN
5,5,01DSRCS6FWFEXE6T3QYZ7QHN7V,Kissy Kissy,4.00098E+11,Baby Girl's Long-Sleeve Cotton Footie,Comfortable cotton footie with scalloped trim,JustKids/Baby024months/InfantGirls/FootiesRompers,2019-11-15 20:41:27.284292+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/kissy-kissy-ba...,Crewneck\nLong sleeves\nFront snap closure\nPi...,"{""Needs Review""}",NaN,NaN
6,6,01DSE9J9B1WCDKEZDBXY9JCD1H,Banana Republic,512628,Untucked Slim-Fit Tech-Stretch Cotton Shirt,"Made with our Tech Stretch Cotton fabric, this...",Unknown,2019-11-11 22:32:50.778391+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,"Made with our Tech Stretch Cotton fabric, this...","{""Needs Review""}",NaN,NaN
7,7,01DSEAZS5R8HJBGWY3XHJSDC1W,Banana Republic,402448,Classic-Fit Bi-Stretch Blazer,Our Classic Fit blazer has a single-button fro...,Unknown,2019-11-11 22:57:41.549451+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Our Classic Fit blazer has a single-button fro...,"{""Needs Review""}",NaN,NaN
8,8,01DSE9HTZN1C0YDB1W421K6TY0,Banana Republic,517546,Print Puff-Sleeve Wrap Dress,Add a little feminine flair to your look with ...,Unknown,2019-11-11 22:32:36.077057+00,2019-12-19 20:40:30.786144+00,NaN,https://bananarepublic.gap.com/browse/product....,Add a little feminine flair to your look with ...,"{""Needs Review""}",NaN,NaN
9,9,01DSJT9E7E6K26W1F0CP4H2E8H,Hunter,4.00011E+11,Hunter x Peppa Pig Print Umbrella,From the Hunter x Peppa Pig Collaboration. Lig...,"JustKids/Girls214/Accessories,JustKids/Boys220...",2019-11-13 16:42:04.392452+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/hunter-hunter-...,Easy-to-hold curved handle\nSafety feature on ...,"{""Needs Review""}",NaN,NaN
